# Evaluate Model Performance on incomplete Stroke Dataset

In [9]:
from read_data import get_dataset
import torch
from torch import nn

Put your batch preprocessing function and model definition below

In [7]:
def unpack(x,max_strock,max_len):
    x_new=torch.zeros(torch.Size([len(x),max_strock,3,max_len]))
    for i,item in enumerate(x):
        for j,strock in enumerate(item):
            #print(strock)
            
            #print(len(strock[0]))
            strock=torch.Tensor(strock)
            #print(x_new[i,j,0,:len(strock[0])].shape)
            #print(strock[0].shape)
            x_new[i,j,0,:len(strock[0])]=strock[0]
            x_new[i,j,1,:len(strock[0])]=strock[1]
        x_new[i,0,2,0]=len(item)
    return x_new
max_strock,max_len=30,200

class CNN(nn.Module):
    def __init__(self,output_size):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(
                in_channels=3,      # input height
                out_channels=6,    # n_filters
                kernel_size=3      # filter size
            ),
            nn.ReLU(),    # activation
            nn.MaxPool1d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
             nn.Conv1d(6,3, 3),
             nn.ReLU(),  # activation
             nn.MaxPool1d(kernel_size=2)
         )
        self.out = nn.Linear(144, output_size)
        #self.out2 = nn.Linear(3*(max_len), output_size)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        #x=self.conv3(x)
        x = x.view(x.size(0), -1)
        #print(x.shape)
        output = self.out(x)
        #output=self.out2(x)
        return output
    
class CRNN(nn.Module):
    def __init__(self):
        super(CRNN, self).__init__()
        self.middle_size=middle_size=40
        self.gru=nn.GRU(input_size=middle_size,hidden_size=5,num_layers=2,bias=False)
        # self.gru=nn.GRU(input_size=3,hidden_size=5,num_layers=1,bias=True)
        self.cnn=CNN(output_size=middle_size)
        self.out=nn.Softmax(dim=-1)
    def forward(self, x):
        x=x.view(-1,3,max_len)
        
        #print(x.shape)
        xx = self.cnn(x)
        xx=xx.view(-1,max_strock,self.middle_size).transpose(0,1)
        output,_ = self.gru(xx)
        #print(output.shape)
        to_send=torch.Tensor(torch.Size([xx.shape[1],5])).cuda()
        for i in range(xx.shape[1]):
            to_send[i]=output[int(x[i*max_strock,2,0])-1,i]
        output = self.out(to_send)
        return output

In [3]:
model_name='CRNN_91.9_2C1D1R.pkl'
model=torch.load(model_name)

Function for calculating accuracy score.

Before use this function, change the batch preprocessing method

In [4]:
def get_acc(model,test_X,test_Y):
    cur_len=0
    acc=0
    batch_size=512
    while cur_len<len(test_X):
        model.zero_grad()
        minibatch_X=test_X[cur_len:cur_len+batch_size]
        #Minibatch preprocessing method here
        minibatch_X=unpack(minibatch_X,max_strock,max_len).cuda()
        #---------------
        minibatch_Y=test_Y[cur_len:cur_len+batch_size]
        minibatch_Y=torch.LongTensor(minibatch_Y).cuda()
        y_predict=model(minibatch_X)
        y_predict=torch.argmax(y_predict,dim=1)
        cur_len+=batch_size
        acc+=(y_predict==minibatch_Y).sum().item()
    return acc/cur_len

In [8]:
def incomplete_test(model,r):
    _,_,test_X,testY=get_dataset(dataset_name='1102_05b633244',test_r=r)
    print("{},{}".format(r,get_acc(model,test_X,testY)))
for r in [0.5,0.7,0.9,1]:
    incomplete_test(model,r)

/raid5/liuchang/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


0.5,0.7632387222782258
0.7,0.8202731224798387
0.9,0.8589654737903226
1,0.9177718623991935
